In [1]:
%matplotlib inline

from __future__ import print_function
from __future__ import division

import os

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook


# let's not pollute this blog post with warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
nest_counts = pd.read_csv(os.path.join('data', 'training_set_nest_counts.csv'),index_col=[0,1])

In [3]:
nest_counts.head()

1895  1903  1908  1921  1947  1956  1957  1959  \
site_id common_name                                                         
ACUN    adelie penguin      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
        chinstrap penguin   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
ADAM    adelie penguin      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
ADAR    adelie penguin      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
AILS    chinstrap penguin   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

                           1960  1964    ...       2004  2005  2006  2007  \
site_id common_name                      ...                                
ACUN    adelie penguin      NaN   NaN    ...     1880.0   NaN   NaN   NaN   
        chinstrap penguin   NaN   NaN    ...        NaN   NaN   NaN   NaN   
ADAM    adelie penguin      NaN   NaN    ...        NaN   NaN   NaN   NaN   
ADAR    adelie penguin      NaN   NaN    ...        NaN   NaN   NaN   NaN   
AILS    chinstrap penguin   NaN   NaN    ...        NaN   NaN   NaN   NaN   

                           2008  2009      2010  2011  2012      2013  
site_id common_name                                                    
ACUN    adelie penguin      NaN   NaN    3079.0   NaN   NaN       NaN  
        chinstrap penguin   NaN   NaN       NaN   NaN   NaN       NaN  
ADAM    adelie penguin      NaN  76.0       NaN   NaN   NaN       NaN  
ADAR    adelie penguin      NaN   NaN  338231.0   NaN   NaN  428516.0  
AILS    chinstrap penguin   NaN   NaN       NaN   NaN   NaN       NaN  

[5 rows x 55 columns]

In [4]:
def preprocess_timeseries(timeseries, first_year, fillna_value=0,en=0):
    """ Takes one of the timeseries dataframes, removes
        columns before `first_year`, and fills NaN values
        with the preceeding value. Then backfills any
        remaining NaNs.
        
        As a courtesy, also turns year column name into
        integers for easy comparisons.
    """
    # column type
    timeseries.columns = timeseries.columns.astype(int)
    
    # subset to just data after first_year
    timeseries = timeseries.loc[:, timeseries.columns >= first_year]
    
    if (en == 1):
        # Forward fill count values. This is a strong assumption.
        timeseries.fillna(method="ffill", axis=1, inplace=True)
        timeseries.fillna(method="bfill", axis=1, inplace=True)

        # For sites with no observations, fill with fill_na_value
        timeseries.fillna(fillna_value, inplace=True)
                 
    return timeseries

In [5]:
nest_counts = preprocess_timeseries(nest_counts,
                                    1980,
                                    fillna_value=0.0,en=0)
nest_counts.head()

1980  1981  1982      1983  1984  1985      1986  \
site_id common_name                                                           
ACUN    adelie penguin      NaN   NaN   NaN       NaN   NaN   NaN       NaN   
        chinstrap penguin   NaN   NaN   NaN    4000.0   NaN   NaN       NaN   
ADAM    adelie penguin      NaN   NaN   NaN       NaN   NaN   NaN       NaN   
ADAR    adelie penguin      NaN   NaN   NaN  256806.0   NaN   NaN  282307.0   
AILS    chinstrap penguin   NaN   NaN   NaN    6000.0   NaN   NaN       NaN   

                           1987      1988  1989    ...       2004  2005  2006  \
site_id common_name                                ...                          
ACUN    adelie penguin      NaN       NaN   NaN    ...     1880.0   NaN   NaN   
        chinstrap penguin   NaN       NaN   NaN    ...        NaN   NaN   NaN   
ADAM    adelie penguin      NaN       NaN   NaN    ...        NaN   NaN   NaN   
ADAR    adelie penguin      NaN  272338.0   NaN    ...        NaN   NaN   NaN   
AILS    chinstrap penguin   NaN       NaN   NaN    ...        NaN   NaN   NaN   

                           2007  2008  2009      2010  2011  2012      2013  
site_id common_name                                                          
ACUN    adelie penguin      NaN   NaN   NaN    3079.0   NaN   NaN       NaN  
        chinstrap penguin   NaN   NaN   NaN       NaN   NaN   NaN       NaN  
ADAM    adelie penguin      NaN   NaN  76.0       NaN   NaN   NaN       NaN  
ADAR    adelie penguin      NaN   NaN   NaN  338231.0   NaN   NaN  428516.0  
AILS    chinstrap penguin   NaN   NaN   NaN       NaN   NaN   NaN       NaN  

[5 rows x 34 columns]

In [6]:
fillna_value = 0.0

In [7]:
def modify_per_row(ts):
    ts_interpolate = ts.interpolate(method = 'linear')
    ts_interpolate.fillna(method="bfill",inplace=True)
    ts_interpolate.fillna(fillna_value, inplace=True)
    return ts_interpolate

In [8]:
interpolated_values = []

for row in nest_counts.iterrows():
    row_interpolate = modify_per_row(row[1])
    interpolated_values.append(row_interpolate.values)

interpolated_df = pd.DataFrame(interpolated_values,
                             index=nest_counts.index,
                             columns=nest_counts.columns) 

In [9]:
interpolated_df.head()

1980      1981      1982      1983  \
site_id common_name                                                 
ACUN    adelie penguin       2008.0    2008.0    2008.0    2008.0   
        chinstrap penguin    4000.0    4000.0    4000.0    4000.0   
ADAM    adelie penguin         76.0      76.0      76.0      76.0   
ADAR    adelie penguin     256806.0  256806.0  256806.0  256806.0   
AILS    chinstrap penguin    6000.0    6000.0    6000.0    6000.0   

                                    1984           1985      1986      1987  \
site_id common_name                                                           
ACUN    adelie penguin       2008.000000    2008.000000    2008.0    2008.0   
        chinstrap penguin    4000.000000    4000.000000    4000.0    4000.0   
ADAM    adelie penguin         76.000000      76.000000      76.0      76.0   
ADAR    adelie penguin     265306.333333  273806.666667  282307.0  277322.5   
AILS    chinstrap penguin    6000.000000    6000.000000    6000.0    6000.0   

                               1988      1989    ...         2004  \
site_id common_name                              ...                
ACUN    adelie penguin       2008.0    2008.0    ...       1880.0   
        chinstrap penguin    4000.0    4000.0    ...       4000.0   
ADAM    adelie penguin         76.0      76.0    ...         76.0   
ADAR    adelie penguin     272338.0  220769.0    ...     338504.0   
AILS    chinstrap penguin    6000.0    6000.0    ...       6000.0   

                                    2005           2006      2007  \
site_id common_name                                                 
ACUN    adelie penguin       2079.833333    2279.666667    2479.5   
        chinstrap penguin    4000.000000    4000.000000    4000.0   
ADAM    adelie penguin         76.000000      76.000000      76.0   
ADAR    adelie penguin     338458.500000  338413.000000  338367.5   
AILS    chinstrap penguin    6000.000000    6000.000000    6000.0   

                                    2008           2009      2010      2011  \
site_id common_name                                                           
ACUN    adelie penguin       2679.333333    2879.166667    3079.0    3079.0   
        chinstrap penguin    4000.000000    4000.000000    4000.0    4000.0   
ADAM    adelie penguin         76.000000      76.000000      76.0      76.0   
ADAR    adelie penguin     338322.000000  338276.500000  338231.0  368326.0   
AILS    chinstrap penguin    6000.000000    6000.000000    6000.0    6000.0   

                               2012      2013  
site_id common_name                            
ACUN    adelie penguin       3079.0    3079.0  
        chinstrap penguin    4000.0    4000.0  
ADAM    adelie penguin         76.0      76.0  
ADAR    adelie penguin     398421.0  428516.0  
AILS    chinstrap penguin    6000.0    6000.0  

[5 rows x 34 columns]

In [10]:
nest_counts2 = interpolated_df

In [11]:
e_n_values = pd.read_csv(
                 os.path.join('data', 'training_set_e_n.csv'),
                 index_col=[0,1]
             )

# Process error data to match our nest_counts data
e_n_values = preprocess_timeseries(e_n_values, 1980, fillna_value=0.05,en=1)
e_n_values.head()

1980  1981  1982  1983  1984  1985  1986  1987  \
site_id common_name                                                         
ACUN    adelie penguin     0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05   
        chinstrap penguin  0.50  0.50  0.50  0.50  0.50  0.50  0.50  0.50   
ADAM    adelie penguin     0.90  0.90  0.90  0.90  0.90  0.90  0.90  0.90   
ADAR    adelie penguin     0.10  0.10  0.10  0.10  0.10  0.10  0.10  0.10   
AILS    chinstrap penguin  0.50  0.50  0.50  0.50  0.50  0.50  0.50  0.50   

                           1988  1989  ...   2004  2005  2006  2007  2008  \
site_id common_name                    ...                                  
ACUN    adelie penguin     0.05  0.05  ...   0.05  0.05  0.05  0.05  0.05   
        chinstrap penguin  0.50  0.50  ...   0.50  0.50  0.50  0.50  0.50   
ADAM    adelie penguin     0.90  0.90  ...   0.90  0.90  0.90  0.90  0.90   
ADAR    adelie penguin     0.10  0.10  ...   0.10  0.10  0.10  0.10  0.10   
AILS    chinstrap penguin  0.50  0.50  ...   0.50  0.50  0.50  0.50  0.50   

                           2009  2010  2011  2012  2013  
site_id common_name                                      
ACUN    adelie penguin     0.05   0.9   0.9   0.9   0.9  
        chinstrap penguin  0.50   0.5   0.5   0.5   0.5  
ADAM    adelie penguin     0.90   0.9   0.9   0.9   0.9  
ADAR    adelie penguin     0.10   0.9   0.9   0.9   0.1  
AILS    chinstrap penguin  0.50   0.5   0.5   0.5   0.5  

[5 rows x 34 columns]

In [12]:
def amape(y_true, y_pred, accuracies):
    """ Adjusted MAPE
    """
    not_nan_mask = ~np.isnan(y_true)
    
    # calculate absolute error
    abs_error = (np.abs(y_true[not_nan_mask] - y_pred[not_nan_mask]))
    
    # calculate the percent error (replacing 0 with 1
    # in order to avoid divide-by-zero errors).
    pct_error = abs_error / np.maximum(1, y_true[not_nan_mask])
    
    # adjust error by count accuracies
    adj_error = pct_error / accuracies[not_nan_mask]
    
    # return the mean as a percentage
    return np.mean(adj_error)

In [13]:
# Let's confirm the best possible score is 0!
amape(nest_counts2.values,
      nest_counts2.values,
      e_n_values.values)

0.0

In [14]:
from sklearn.ensemble import RandomForestRegressor

def train_model_per_row(ts, acc, split_year=2010):
    # Split into train/test to tune our parameter
    train = ts.iloc[ts.index < split_year]
    
    test = ts.iloc[ts.index >= split_year]
    test_acc = acc.iloc[acc.index >= split_year]
    
    # Store best lag parameter
    best_mape = np.inf 
    best_lag = None
    best_n_estimators = None
  
    # Test linear regression models with the most recent
    # 2 points through using all of the points
    list1 = [10]
    for estimators in (list1):
        for lag in range(2, train.shape[0]):
            # fit the model
            temp_model = RandomForestRegressor(n_estimators = estimators,random_state = 13)
            temp_model.fit(
                train.index[-lag:].values.reshape(-1, 1),
                train[-lag:]
            )

            # make our predictions on the test set
            preds = temp_model.predict(
                        test.index.values.reshape(-1, 1)
                    )

            # calculate the score using the custom metric
            mape = amape(test.values,
                         preds,
                         test_acc.values)

            # if it's the best score yet, hold on to the parameter
            if mape < best_mape:
                best_mape = mape
                best_lag = lag
                best_n_estimators = estimators

    # return model re-trained on entire dataset
    final_model = RandomForestRegressor(n_estimators = best_n_estimators,random_state = 13)
    final_model.fit(ts.index[-best_lag:].values.reshape(-1, 1),ts[-best_lag:])
    
    return final_model,best_mape

In [15]:
models = {}
best_mape = {}

for i, row in tqdm_notebook(nest_counts2.iterrows(),
                            total=nest_counts2.shape[0]):
    acc = e_n_values.loc[i]
    models[i],best_mape[i] = train_model_per_row(row, acc)

In [16]:
errors_sum = sum(best_mape.values())

In [17]:
errors_sum

2230.1950933922367

In [18]:
submission_format = pd.read_csv(
    os.path.join('data','submission_format.csv'),
    index_col=[0, 1]
)

print(submission_format.shape)
submission_format.head()

(648, 4)


2014  2015  2016  2017
site_id common_name                              
ACUN    adelie penguin      0.0   0.0   0.0   0.0
        chinstrap penguin   0.0   0.0   0.0   0.0
ADAM    adelie penguin      0.0   0.0   0.0   0.0
ADAR    adelie penguin      0.0   0.0   0.0   0.0
AILS    chinstrap penguin   0.0   0.0   0.0   0.0

In [19]:
preds = []

# For every row in the submission file
for i, row in tqdm_notebook(submission_format.iterrows(),
                            total=submission_format.shape[0]):
    
    # get the model for this site + common_name
    model = models[i]
    
    # make predictions using the model
    row_predictions = model.predict(
        submission_format.columns.values.reshape(-1, 1)
    )
       
    # keep our predictions, rounded to nearest whole number
    preds.append(np.round(row_predictions))

# Create a dataframe that we can write out to a CSV
prediction_df = pd.DataFrame(preds,
                             index=submission_format.index,
                             columns=submission_format.columns)

prediction_df.head()

2014      2015      2016      2017
site_id common_name                                              
ACUN    adelie penguin       3079.0    3079.0    3079.0    3079.0
        chinstrap penguin    4000.0    4000.0    4000.0    4000.0
ADAM    adelie penguin         76.0      76.0      76.0      76.0
ADAR    adelie penguin     401435.0  401435.0  401435.0  401435.0
AILS    chinstrap penguin    6000.0    6000.0    6000.0    6000.0

In [20]:
prediction_df.to_csv('predictionsRFInterpolationWithSeed.csv')

In [21]:
!head -n 5 predictionsRFInterpolationWithSeed.csv

site_id,common_name,2014,2015,2016,2017
ACUN,adelie penguin,3079.0,3079.0,3079.0,3079.0
ACUN,chinstrap penguin,4000.0,4000.0,4000.0,4000.0
ADAM,adelie penguin,76.0,76.0,76.0,76.0
ADAR,adelie penguin,401435.0,401435.0,401435.0,401435.0
